In [ ]:
# %%
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import display


# --------------------------------------------------------------
# Load data
# --------------------------------------------------------------
df = pd.read_pickle("../../data/interim/01_data_processed.pkl")

# --------------------------------------------------------------
# Plot single columns
# --------------------------------------------------------------
set_df = df[df["set"] == 1]
plt.plot(set_df["acc_y"])
# plt.show()
plt.plot(set_df["acc_y"].reset_index(drop=True))
# plt.show()
# --------------------------------------------------------------
# Plot all exercises
# --------------------------------------------------------------
for label in df["label"].unique():
    subset = df[df["label"] == label]
    fig, ax = plt.subplots()
    plt.plot(subset["acc_y"].reset_index(drop=True),label = label)
    plt.legend()
    plt.show()

for label in df["label"].unique():
    subset = df[df["label"] == label]
    fig, ax = plt.subplots()
    plt.plot(subset[:100]["acc_y"].reset_index(drop=True),label = label)
    plt.legend()
    plt.show()
# --------------------------------------------------------------
# Adjust plot settings
# --------------------------------------------------------------
mpl.style.use("seaborn-v0_8-deep")
mpl.rcParams["figure.figsize"] = (13,5)
mpl.rcParams["figure.dpi"] = 100
# --------------------------------------------------------------
# Compare medium vs. heavy sets
# --------------------------------------------------------------
# ở lúc squat trung bình thì là  gia tốc màu xanh lá  cao hơn thì tức là người ta hạ xuống tạ nhanh hơn và đứng dậy cũng nhanh hơn
# ngược lại ở mức nặng thì gia tốc xanh dương thấp hơn tức là người ta hạ tạ xuống chậm và đứng dậy chậm hơn
category_df = df.query("label == 'squat'").query("participant == 'A'").reset_index()
fig, ax = plt.subplots()
category_df.groupby(["category"])["acc_y"].plot()
ax.set_ylabel("acc_y")
ax.set_xlabel("samples")
plt.legend()
plt.show()
# --------------------------------------------------------------
# Compare participants : kiểm tra người thực hiện ví dụ a có giống người b không 
# --------------------------------------------------------------
# qua dữ liệu này ta thấy người E tập nhiều bài ép tạ hơn 
# thêm sort_value để sắp xếp lại chứ không sẽ bị lộn xộn 
participant_df = df.query("label == 'bench'").sort_values("participant").reset_index()
fig, ax = plt.subplots()
participant_df.groupby(["participant"])["acc_y"].plot()
ax.set_ylabel("acc_y")
ax.set_xlabel("samples")
plt.legend()
plt.show()
# --------------------------------------------------------------
# Plot multiple axis
# --------------------------------------------------------------
label = "squat"
participant = "A"
all_axis_df = df.query(f"label == '{label}'").query(f"participant == '{participant}'").reset_index()

fig, ax = plt.subplots()
all_axis_df[["acc_x" , "acc_y", "acc_z"]].plot(ax=ax)
ax.set_ylabel("acc_y")
ax.set_xlabel("samples")
plt.legend()
plt.show()
# --------------------------------------------------------------
# Create a loop to plot all combinations per sensor
# --------------------------------------------------------------
labels = df["label"].unique()
participants = df["participant"].unique()

for label in labels:
    for participant in participants: 
        all_axis_df = (df.query(f"label == '{label}'").query(f"participant == '{participant}'").reset_index())

        if len(all_axis_df) > 0 :
            fig, ax = plt.subplots()
            all_axis_df[["acc_x" , "acc_y", "acc_z"]].plot(ax=ax)
            ax.set_ylabel("acc_y")
            ax.set_xlabel("samples")
            plt.title(f"{label} ({participant})".title())
            plt.legend()
            plt.show()

for label in labels:
    for participant in participants: 
        all_axis_df = (df.query(f"label == '{label}'").query(f"participant == '{participant}'").reset_index())

        if len(all_axis_df) > 0 :
            fig, ax = plt.subplots()
            all_axis_df[["gyr_x" , "gyr_y", "gyr_z"]].plot(ax=ax)
            ax.set_ylabel("gyr_y")
            ax.set_xlabel("samples")
            plt.title(f"{label} ({participant})".title())
            plt.legend()
            plt.show()
        
# --------------------------------------------------------------
# Combine plots in one figure
# --------------------------------------------------------------
label = "row"
participant = "A"
combined_plot_df = (
    df.query(f"label == '{label}'")
    .query(f"participant == '{participant}'")
    .reset_index(drop = True)
)
fig , ax = plt.subplots(nrows=2, sharex=True, figsize=(13,5))
combined_plot_df[["acc_x" , "acc_y", "acc_z"]].plot(ax=ax[0])
combined_plot_df[["gyr_x" , "gyr_y", "gyr_z"]].plot(ax=ax[1])

ax[0].legend(loc="upper center", bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, shadow=True)
ax[1].legend(loc="upper center", bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, shadow=True)
ax[1].set_xlabel("Samples")
plt.show()
# --------------------------------------------------------------
# Loop over all combinations and export for both sensors 
# tạo biểu đồ kết hợp giữa gia tốc kế với con quay hồi chuyển 
# --------------------------------------------------------------
labels = df["label"].unique()
participants = df["participant"].unique()

for label in labels:
    for participant in participants: 
        combined_plot_df = (df.query(f"label == '{label}'").query(f"participant == '{participant}'").reset_index())

        if len(combined_plot_df) > 0 :
            fig , ax = plt.subplots(nrows=2, sharex=True, figsize=(13,5))
            combined_plot_df[["acc_x" , "acc_y", "acc_z"]].plot(ax=ax[0])
            combined_plot_df[["gyr_x" , "gyr_y", "gyr_z"]].plot(ax=ax[1])

            ax[0].legend(loc="upper center", bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, shadow=True)
            ax[1].legend(loc="upper center", bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, shadow=True)
            ax[1].set_xlabel("Samples")

            plt.savefig(f"../../reports/figures/{label.title()} ({participant}).png")
            plt.show()